# Debug feature engineering

# 1. Imports

## 1.1 Packages

In [1]:
import pandas as pd


## 1.2 Options

## 1.3 Datasets

In [2]:
# dict_metadata_datasets = pd.read_pickle("../data/02_intermediate/dict_metadata_datasets.pkl")

## 1.4 Functions

In [3]:
from impostor_hunt_in_texts.pipelines.feature_engineering.nodes import (
    extract_features,
    load_model_and_tokenizer,
)
from impostor_hunt_in_texts.utils.utils import load_hf_datasetdict, split_dataset_dict

/Users/benjaminwallyn/Git/impostor-hunt-in-texts/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Feature engineering

In [4]:
dict_metadata_datasets = {"save_path": "../data/02_intermediate/dict_datasets"}

In [5]:
dataset_dict = load_hf_datasetdict(dict_metadata_datasets)
dataset_train, dataset_test = split_dataset_dict(dataset_dict)

In [6]:
model, tokenizer = load_model_and_tokenizer("bert-base-uncased")

In [7]:
dataset_train_features, train_ids = extract_features(
    dataset=dataset_train,
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    stride=216,
    device="cpu",
)

Extracting features: 100%|██████████| 95/95 [00:36<00:00,  2.60it/s]


In [8]:
dataset_train_features

array([[-1.8221563e-01,  1.8503173e-01,  2.6603371e-01, ...,
         1.9781753e-02,  5.7555810e-03,  1.6396128e-03],
       [-1.6375388e-01,  3.2518438e-01,  4.7533748e-01, ...,
         3.5610597e-04, -3.1663883e-03, -7.9679349e-03],
       [-1.8221563e-01,  1.8503173e-01,  2.6603371e-01, ...,
         1.9781753e-02,  5.7555810e-03,  1.6396128e-03],
       ...,
       [-1.6375388e-01,  3.2518438e-01,  4.7533748e-01, ...,
         3.5610597e-04, -3.1663883e-03, -7.9679349e-03],
       [-1.6375388e-01,  3.2518438e-01,  4.7533748e-01, ...,
         3.5610597e-04, -3.1663883e-03, -7.9679349e-03],
       [-1.8221563e-01,  1.8503173e-01,  2.6603371e-01, ...,
         1.9781753e-02,  5.7555810e-03,  1.6396128e-03]],
      shape=(95, 3072), dtype=float32)